In [10]:
pip install pyspark


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, rank
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, LongType, DoubleType, StringType



spark = SparkSession.builder \
    .appName("Health & Fitness Tracker") \
    .getOrCreate()


schema = StructType([
    StructField("user_id", LongType(), True),
    StructField("date", StringType(), True),
    StructField("steps", LongType(), True),
    StructField("calories_burned", LongType(), True),  
    StructField("hours_of_sleep", DoubleType(), True),
    StructField("workout_type", StringType(), True)
])


fitness_data = [
    (1, '2023-09-01', 12000, 500, 7.0, 'Cardio'),
    (2, '2023-09-01', 8000, 400, 6.5, 'Strength'),
    (3, '2023-09-01', 15000, 650, 8.0, 'Yoga'),
    (1, '2023-09-02', 10000, 450, 6.0, 'Cardio'),
    (2, '2023-09-02', 9500, 500, 7.0, 'Cardio'),
    (3, '2023-09-02', 14000, 600, 7.5, 'Strength'),
    (1, '2023-09-03', 13000, 550, 8.0, 'Yoga'),
    (2, '2023-09-03', 12000, 520, 6.5, 'Yoga'),
    (3, '2023-09-03', 16000, 700, 7.0, 'Cardio')
]


fitness_df = spark.createDataFrame(fitness_data, schema)

fitness_df.show()


+-------+----------+-----+---------------+--------------+------------+
|user_id|      date|steps|calories_burned|hours_of_sleep|workout_type|
+-------+----------+-----+---------------+--------------+------------+
|      1|2023-09-01|12000|            500|           7.0|      Cardio|
|      2|2023-09-01| 8000|            400|           6.5|    Strength|
|      3|2023-09-01|15000|            650|           8.0|        Yoga|
|      1|2023-09-02|10000|            450|           6.0|      Cardio|
|      2|2023-09-02| 9500|            500|           7.0|      Cardio|
|      3|2023-09-02|14000|            600|           7.5|    Strength|
|      1|2023-09-03|13000|            550|           8.0|        Yoga|
|      2|2023-09-03|12000|            520|           6.5|        Yoga|
|      3|2023-09-03|16000|            700|           7.0|      Cardio|
+-------+----------+-----+---------------+--------------+------------+



In [17]:
# 1. Find the Total Steps Taken by Each User
total_steps_per_user = fitness_df.groupBy("user_id").agg(sum("steps").alias("total_steps"))
total_steps_per_user.show()

# 2. Filter Days with More Than 10,000 Steps
days_gt_10000_steps = fitness_df.filter(col("steps") > 10000)
days_gt_10000_steps.show()

# 3. Calculate the Average Calories Burned by Workout Type
avg_calories_by_workout = fitness_df.groupBy("workout_type").agg(avg("calories_burned").alias("avg_calories_burned"))
avg_calories_by_workout.show()

# 4. Identify the Day with the Most Steps for Each User
window_spec = Window.partitionBy("user_id").orderBy(col("total_steps").desc())
most_steps_per_day = fitness_df.groupBy("user_id", "date").agg(sum("steps").alias("total_steps")) \
    .withColumn("rank", rank().over(window_spec)) \
    .filter(col("rank") == 1)

# Show the result
most_steps_per_day.show()
# 5. Find Users Who Burned More Than 600 Calories on Any Day
users_burned_600_calories = fitness_df.filter(col("calories_burned") > 600) \
    .select("user_id").distinct()
users_burned_600_calories.show()

# 6. Calculate the Average Hours of Sleep per User
avg_sleep_per_user = fitness_df.groupBy("user_id").agg(avg("hours_of_sleep").alias("avg_hours_of_sleep"))
avg_sleep_per_user.show()

# 7. Find the Total Calories Burned per Day
total_calories_per_day = fitness_df.groupBy("date").agg(sum("calories_burned").alias("total_calories_burned"))
total_calories_per_day.show()

# 8. Identify Users Who Did Different Types of Workouts
users_multiple_workouts = fitness_df.groupBy("user_id").agg(countDistinct("workout_type").alias("workout_types_count")) \
    .filter(col("workout_types_count") > 1)
users_multiple_workouts.show()

# 9. Calculate the Total Number of Workouts per User
total_workouts_per_user = fitness_df.groupBy("user_id").agg(count("workout_type").alias("total_workouts"))
total_workouts_per_user.show()

# 10. Create a New Column for "Active" Days
df_with_activity = fitness_df.withColumn("active_day", when(col("steps") > 10000, "Active").otherwise("Inactive"))
df_with_activity.show()


+-------+-----------+
|user_id|total_steps|
+-------+-----------+
|      1|      35000|
|      3|      45000|
|      2|      29500|
+-------+-----------+

+-------+----------+-----+---------------+--------------+------------+
|user_id|      date|steps|calories_burned|hours_of_sleep|workout_type|
+-------+----------+-----+---------------+--------------+------------+
|      1|2023-09-01|12000|            500|           7.0|      Cardio|
|      3|2023-09-01|15000|            650|           8.0|        Yoga|
|      3|2023-09-02|14000|            600|           7.5|    Strength|
|      1|2023-09-03|13000|            550|           8.0|        Yoga|
|      2|2023-09-03|12000|            520|           6.5|        Yoga|
|      3|2023-09-03|16000|            700|           7.0|      Cardio|
+-------+----------+-----+---------------+--------------+------------+

+------------+-------------------+
|workout_type|avg_calories_burned|
+------------+-------------------+
|    Strength|              5

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("Music Streaming") \
    .getOrCreate()

music_data = [
    (1, 'Blinding Lights', 'The Weeknd', 200, '2023-09-01 08:15:00', 'New York'),
    (2, 'Shape of You', 'Ed Sheeran', 240, '2023-09-01 09:20:00', 'Los Angeles'),
    (3, 'Levitating', 'Dua Lipa', 180, '2023-09-01 10:30:00', 'London'),
    (1, 'Starboy', 'The Weeknd', 220, '2023-09-01 11:00:00', 'New York'),
    (2, 'Perfect', 'Ed Sheeran', 250, '2023-09-01 12:15:00', 'Los Angeles'),
    (3, 'Don\'t Start Now', 'Dua Lipa', 200, '2023-09-02 08:10:00', 'London'),
    (1, 'Save Your Tears', 'The Weeknd', 210, '2023-09-02 09:00:00', 'New York'),
    (2, 'Galway Girl', 'Ed Sheeran', 190, '2023-09-02 10:00:00', 'Los Angeles'),
    (3, 'New Rules', 'Dua Lipa', 230, '2023-09-02 11:00:00', 'London')
]
music_columns = ["user_id", "song_title", "artist", "duration_seconds", "streaming_time", "location"]
music_df = spark.createDataFrame(music_data, music_columns)
music_df.show()


+-------+---------------+----------+----------------+-------------------+-----------+
|user_id|     song_title|    artist|duration_seconds|     streaming_time|   location|
+-------+---------------+----------+----------------+-------------------+-----------+
|      1|Blinding Lights|The Weeknd|             200|2023-09-01 08:15:00|   New York|
|      2|   Shape of You|Ed Sheeran|             240|2023-09-01 09:20:00|Los Angeles|
|      3|     Levitating|  Dua Lipa|             180|2023-09-01 10:30:00|     London|
|      1|        Starboy|The Weeknd|             220|2023-09-01 11:00:00|   New York|
|      2|        Perfect|Ed Sheeran|             250|2023-09-01 12:15:00|Los Angeles|
|      3|Don't Start Now|  Dua Lipa|             200|2023-09-02 08:10:00|     London|
|      1|Save Your Tears|The Weeknd|             210|2023-09-02 09:00:00|   New York|
|      2|    Galway Girl|Ed Sheeran|             190|2023-09-02 10:00:00|Los Angeles|
|      3|      New Rules|  Dua Lipa|             230|2

In [20]:
# 1. Calculate the Total Listening Time for Each User
total_listening_time_per_user = music_df.groupBy("user_id").agg(sum("duration_seconds").alias("total_listening_time"))
total_listening_time_per_user.show()

# 2. Filter Songs Streamed for More Than 200 Seconds
songs_gt_200_seconds = music_df.filter(col("duration_seconds") > 200)
songs_gt_200_seconds.show()

# 3. Find the Most Popular Artist (by Total Streams)
most_popular_artist = music_df.groupBy("artist").agg(count("song_title").alias("total_streams")) \
    .orderBy(col("total_streams").desc()).first()
print(f"Most Popular Artist: {most_popular_artist}")

# 4. Identify the Song with the Longest Duration
longest_song = music_df.orderBy(col("duration_seconds").desc()).first()
print(f"Longest Song: {longest_song}")

# 5. Calculate the Average Song Duration by Artist
avg_duration_by_artist = music_df.groupBy("artist").agg(avg("duration_seconds").alias("avg_duration_seconds"))
avg_duration_by_artist.show()

# 6. Find the Top 3 Most Streamed Songs per User
top_songs_per_user = music_df.groupBy("user_id", "song_title") \
    .agg(count("song_title").alias("stream_count")) \
    .withColumn("rank", rank().over(Window.partitionBy("user_id").orderBy(col("stream_count").desc()))) \
    .filter(col("rank") <= 3)
top_songs_per_user.show()

# 7. Calculate the Total Number of Streams per Day
streams_per_day = music_df.groupBy(music_df.streaming_time.substr(1, 10).alias("date")) \
    .agg(count("song_title").alias("total_streams"))
streams_per_day.show()

# 8. Identify Users Who Streamed Songs from More Than One Artist
users_multiple_artists = music_df.groupBy("user_id").agg(countDistinct("artist").alias("artist_count")) \
    .filter(col("artist_count") > 1)
users_multiple_artists.show()

# 9. Calculate the Total Streams for Each Location
streams_per_location = music_df.groupBy("location").agg(count("song_title").alias("total_streams"))
streams_per_location.show()

# 10. Create a New Column to Classify Long and Short Songs
df_with_song_length = music_df.withColumn("song_length", when(col("duration_seconds") > 200, "Long").otherwise("Short"))
df_with_song_length.show()

+-------+--------------------+
|user_id|total_listening_time|
+-------+--------------------+
|      1|                 630|
|      3|                 610|
|      2|                 680|
+-------+--------------------+

+-------+---------------+----------+----------------+-------------------+-----------+
|user_id|     song_title|    artist|duration_seconds|     streaming_time|   location|
+-------+---------------+----------+----------------+-------------------+-----------+
|      2|   Shape of You|Ed Sheeran|             240|2023-09-01 09:20:00|Los Angeles|
|      1|        Starboy|The Weeknd|             220|2023-09-01 11:00:00|   New York|
|      2|        Perfect|Ed Sheeran|             250|2023-09-01 12:15:00|Los Angeles|
|      1|Save Your Tears|The Weeknd|             210|2023-09-02 09:00:00|   New York|
|      3|      New Rules|  Dua Lipa|             230|2023-09-02 11:00:00|     London|
+-------+---------------+----------+----------------+-------------------+-----------+

Most Po

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("Retail Store Sales") \
    .getOrCreate()

retail_data = [
    (1, 'Apple', 'Groceries', 0.50, 10, '2023-09-01'),
    (2, 'T-shirt', 'Clothing', 15.00, 2, '2023-09-01'),
    (3, 'Notebook', 'Stationery', 2.00, 5, '2023-09-02'),
    (4, 'Banana', 'Groceries', 0.30, 12, '2023-09-02'),
    (5, 'Laptop', 'Electronics', 800.00, 1, '2023-09-03'),
    (6, 'Pants', 'Clothing', 25.00, 3, '2023-09-03'),
    (7, 'Headphones', 'Electronics', 100.00, 2, '2023-09-04'),
    (8, 'Pen', 'Stationery', 1.00, 10, '2023-09-04'),
    (9, 'Orange', 'Groceries', 0.60, 8, '2023-09-05'),
    (10, 'Sneakers', 'Clothing', 50.00, 1, '2023-09-05')
]
retail_columns = ["transaction_id", "product_name", "category", "price", "quantity", "sales_date"]
retail_df = spark.createDataFrame(retail_data, retail_columns)
retail_df.show()

+--------------+------------+-----------+-----+--------+----------+
|transaction_id|product_name|   category|price|quantity|sales_date|
+--------------+------------+-----------+-----+--------+----------+
|             1|       Apple|  Groceries|  0.5|      10|2023-09-01|
|             2|     T-shirt|   Clothing| 15.0|       2|2023-09-01|
|             3|    Notebook| Stationery|  2.0|       5|2023-09-02|
|             4|      Banana|  Groceries|  0.3|      12|2023-09-02|
|             5|      Laptop|Electronics|800.0|       1|2023-09-03|
|             6|       Pants|   Clothing| 25.0|       3|2023-09-03|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|
|             8|         Pen| Stationery|  1.0|      10|2023-09-04|
|             9|      Orange|  Groceries|  0.6|       8|2023-09-05|
|            10|    Sneakers|   Clothing| 50.0|       1|2023-09-05|
+--------------+------------+-----------+-----+--------+----------+



In [28]:
# 1. Calculate the Total Revenue per Category
retail_df = retail_df.withColumn("total_sales", col("price") * col("quantity"))
total_revenue_per_category = retail_df.groupBy("category").agg(sum("total_sales").alias("total_revenue"))
total_revenue_per_category.show()

# 2. Filter Transactions Where the Total Sales Amount is Greater Than $100
transactions_gt_100 = retail_df.filter(col("total_sales") > 100)
transactions_gt_100.show()

# 3. Find the Most Sold Product
most_sold_product = retail_df.groupBy("product_name").agg(sum("quantity").alias("total_quantity")) \
    .orderBy(col("total_quantity").desc()).first()
print(f"Most Sold Product: {most_sold_product}")

# 4. Calculate the Average Price per Product Category
avg_price_per_category = retail_df.groupBy("category").agg(avg("price").alias("avg_price"))
avg_price_per_category.show()

# 5. Find the Top 3 Highest Grossing Products
top_grossing_products = retail_df.groupBy("product_name").agg(sum("total_sales").alias("total_revenue")) \
    .orderBy(col("total_revenue").desc()).limit(3)
top_grossing_products.show()

# 6. Calculate the Total Number of Items Sold per Day
total_items_sold_per_day = retail_df.groupBy("sales_date").agg(sum("quantity").alias("total_items_sold"))
total_items_sold_per_day.show()

# 7. Identify the Product with the Lowest Price in Each Category
lowest_price_per_category = retail_df.groupBy("category").agg(min("price").alias("lowest_price"))

# Join the original DataFrame with the DataFrame containing lowest prices
lowest_price_per_product = retail_df.join(
    lowest_price_per_category,
    on=(retail_df["category"] == lowest_price_per_category["category"]) & (retail_df["price"] == lowest_price_per_category["lowest_price"])
)

lowest_price_per_product.show()

# 8. Calculate the Total Revenue for Each Product
total_revenue_per_product = retail_df.groupBy("product_name").agg(sum("total_sales").alias("total_revenue"))
total_revenue_per_product.show()

# 9. Find the Total Sales per Day for Each Category
sales_per_day_per_category = retail_df.groupBy("sales_date", "category").agg(sum("total_sales").alias("total_sales"))
sales_per_day_per_category.show()

# 10. Create a New Column for Discounted Price
df_with_discount = retail_df.withColumn("discounted_price", col("price") * 0.9)
df_with_discount.show()

+-----------+------------------+
|   category|     total_revenue|
+-----------+------------------+
| Stationery|              20.0|
|  Groceries|13.399999999999999|
|Electronics|            1000.0|
|   Clothing|             155.0|
+-----------+------------------+

+--------------+------------+-----------+-----+--------+----------+-----------+
|transaction_id|product_name|   category|price|quantity|sales_date|total_sales|
+--------------+------------+-----------+-----+--------+----------+-----------+
|             5|      Laptop|Electronics|800.0|       1|2023-09-03|      800.0|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|      200.0|
+--------------+------------+-----------+-----+--------+----------+-----------+

Most Sold Product: Row(product_name='Banana', total_quantity=12)
+-----------+------------------+
|   category|         avg_price|
+-----------+------------------+
| Stationery|               1.5|
|  Groceries|0.4666666666666666|
|Electronics|           